In [1]:
# classification mnist example 
import  os
#os.environ['nvcc_path']='/usr/local/cuda-11.0/bin/nvcc'
import jittor as jt  # 将 jittor 引入
from jittor import nn, Module  # 引入相关的模块
import numpy as np
import math 
from jittor import init
if jt.has_cuda:
    jt.flags.use_cuda = 1 # jt.flags.use_cuda 表示是否使用 gpu 训练。
# 如果 jt.flags.use_cuda=1，表示使用GPU训练 如果 jt.flags.use_cuda = 0 表示使用 CPU
from jittor.dataset.mnist import MNIST 
#由于 MNIST 是一个常见的数据集，其数据载入已经被封装进 jittor 所以可以直接调用。

In [2]:
import gzip
from PIL import Image
from jittor.dataset import Dataset
from jittor_utils.misc import download_url_to_local

class MNIST(Dataset):
    def __init__(self, data_root="./mnist_data/", train=True ,download=True, batch_size=1, shuffle=False):
        # if you want to test resnet etc you should set input_channel = 3, because the net set 3 as the input dimensions
        super().__init__()
        self.data_root = data_root
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.is_train = train
        if download == True:
            self.download_url()

        filesname = [
                "train-images-idx3-ubyte.gz",
                "t10k-images-idx3-ubyte.gz",
                "train-labels-idx1-ubyte.gz",
                "t10k-labels-idx1-ubyte.gz"
        ]
        self.mnist = {}
        if self.is_train:
            with gzip.open(data_root + filesname[0], 'rb') as f:
                self.mnist["images"] = np.frombuffer(f.read(), np.uint8, offset=16).reshape(-1,28, 28)
            with gzip.open(data_root + filesname[2], 'rb') as f:
                self.mnist["labels"] = np.frombuffer(f.read(), np.uint8, offset=8)
        else:
            with gzip.open(data_root + filesname[1], 'rb') as f:
                self.mnist["images"] = np.frombuffer(f.read(), np.uint8, offset=16).reshape(-1,28, 28)
            with gzip.open(data_root + filesname[3], 'rb') as f:
                self.mnist["labels"] = np.frombuffer(f.read(), np.uint8, offset=8)
        assert(self.mnist["images"].shape[0] == self.mnist["labels"].shape[0])
        self.total_len = self.mnist["images"].shape[0]
        # this function must be called
        self.set_attrs(batch_size = self.batch_size, total_len=self.total_len, shuffle= self.shuffle)
    def __getitem__(self, index):
        img = Image.fromarray (self.mnist['images'][index]) 
        img = np.array (img)
        img = img[np.newaxis, :]
        return np.array((img / 255.0), dtype = np.float32), self.mnist['labels'][index]

    def download_url(self):
        resources = [
            ("http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz", "f68b3c2dcbeaaa9fbdd348bbdeb94873"),
            ("http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz", "d53e105ee54ea40749a09fcbcd1e9432"),
            ("http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz", "9fb629c4189551a2d022fa330f9573f3"),
            ("http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz", "ec29112dd5afa0611ce80d1b7f02629c")
        ]

        for url, md5 in resources:
            filename = url.rpartition('/')[2]
            download_url_to_local(url, filename, self.data_root, md5)


In [3]:
class Model (Module):
    def __init__ (self):
        super (Model, self).__init__()

        self.relu = nn.Relu()
        self.fc1 = nn.Linear (28*28, 512)
        self.fc2 = nn.Linear (512, 10)
    def execute (self, x) : 
        # it's simliar to forward function in Pytorch 
        x=jt.reshape(x,[x.shape[0],-1])
        
        x = self.fc1 (x)
        x = self.relu(x)
        x = self.fc2 (x)
        return x


In [4]:
import ipynb_importer
import jittor_metrics as metrics

importing Jupyter notebook from jittor_metrics.ipynb


In [5]:
batch_size = 64
learning_rate = 1e-3
epochs = 20
losses = []
losses_idx = []
train_loader = MNIST(train=True, batch_size=batch_size, shuffle=True)

val_loader = MNIST(train=False, batch_size=1, shuffle=False)

model = Model ()
optimizer = nn.Adam(model.parameters(), learning_rate)

losses=metrics.MetricResultStorage()
metric=metrics.SparseAccuracy()

In [7]:
from tqdm import tqdm
epochs=1
print('start train')
for epoch in range(epochs):
#     metric.reset_states()
    model.train()
    lens = len(train_loader)
    
#     losses = []
#     optimizer.zero_grad()
    with tqdm(total=lens, unit='step',ascii=True) as tq:
        for batch_idx, (x,y) in enumerate(train_loader):
            
            pred = model(x)
            loss = nn.cross_entropy_loss(pred,y)
            
            optimizer.step (loss)
            loss_f=np.sum(loss.numpy())
            losses.push(loss_f)
            acc=metric.update_state(y,pred)
            
            tq.set_postfix(loss='{:.4f}'.format(losses.result),
                           acc='{:.4f}'.format(metric.result),
                          realtime='{:.4f}'.format(loss_f))
            tq.update(1)

start train


 25%|########9                           | 233/938 [00:09<00:27, 25.79step/s, acc=0.8692, loss=0.4808, realtime=0.2451]


KeyboardInterrupt: 